## 뉴스 웹 크롤링 

### 크롤링 대상
1. [네이버 뉴스 경제/금융 카테고리 페이지](https://news.naver.com/breakingnews/section/101/259)
2. [네이버 뉴스 경제/증권 카테고리 페이지](https://news.naver.com/breakingnews/section/101/258)
3. [네이버 뉴스 경제/산업,제계 카테고리 페이지](https://news.naver.com/breakingnews/section/101/261)
4. [네이버 뉴스 경제/중기,벤처 카테고리 페이지](https://news.naver.com/breakingnews/section/101/771)
5. [네이버 뉴스 경제/부동산 카테고리 페이지](https://news.naver.com/breakingnews/section/101/260)
6. [네이버 뉴스 경제/글로벌 경제 카테고리 페이지](https://news.naver.com/breakingnews/section/101/262)
7. [네이버 뉴스 경제/생활 경제 카테고리 페이지](https://news.naver.com/breakingnews/section/101/310)
8. [네이버 뉴스 경제/경제 일반 카테고리 페이지](https://news.naver.com/breakingnews/section/101/263)

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import csv
from datetime import datetime, timedelta
import time

In [45]:
urls = {
    "금융" : "https://news.naver.com/breakingnews/section/101/259",
    "증권" : "https://news.naver.com/breakingnews/section/101/258",
    "산업/재계" : "https://news.naver.com/breakingnews/section/101/261",
    "중기/벤처" : "https://news.naver.com/breakingnews/section/101/771",
    "부동산" : "https://news.naver.com/breakingnews/section/101/260",
    "글로벌 경제" : "https://news.naver.com/breakingnews/section/101/262",
    "생활경제" : "https://news.naver.com/breakingnews/section/101/310",
    "경제일반" : "https://news.naver.com/breakingnews/section/101/263"
    }

In [39]:
urls = {
    "금융" : "https://news.naver.com/breakingnews/section/101/259"
    }

### 뉴스 크롤링 기간 설정
특정 기간 동안의 뉴스를 크롤링하려면 period_days 변수를 사용하여 기간을 설정할 수 있습니다. <br/>
예를 들어, 현재 시점부터 과거 3일 전까지의 기사를 수집하려면, 다음과 같이 설정합니다 :

``` python
days_to_crawl = 3
```

이 변수는 크롤링할 데이터의 기준 기간(일 단위)을 나타내며, 설정한 값에 따라, 현재 시점을 기준으로 며칠 전까지의 기사를 필터링하여 가져옵니다.

유의사항 :
- days_to_crawl의 가능한 값은 1~30으로 설정하는 것이 적절합니다.
- 값이 30을 초과하면 과도한 요청으로 인해 IP 차단이나 접속 거부를 유발할 수 있으니 주의가 필요합니다.

In [24]:
days_to_crawl = 1

In [25]:
# CSV 파일 작성 함수
def save_to_csv(file_name, data, fields):
    """
    데이터를 CSV 파일에 저장하는 함수
    :param file_name: 저장할 CSV 파일 이름
    :param data: 저장할 데이터 (리스트 형식)
    :param fields: CSV 헤더 (리스트 형식)
    """
    with open(file_name, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # 헤더 작성
        writer.writerow(fields)
        # 데이터 작성
        writer.writerows(data)

In [26]:
def initialize_webdriver():
    """WebDriver 초기화."""
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [28]:
def parse_date_string(date_string):
    """
    게시 날짜 문자열을 처리하여 datetime 객체 반환.
    - "1일 전", "2일 전" 등 상대적 표현을 처리.
    - "~시간 전", "~분 전" 등 세부 시간 단위 처리.
    - "YYYY.MM.DD." 형식의 절대 날짜를 처리.
    """
    now = datetime.now()
    
    if "분전" in date_string:
        # "~분전" 처리
        minutes_ago = int(date_string.replace("분전", "").strip())
        return now - timedelta(minutes=minutes_ago)
    elif "시간전" in date_string:
        # "~시간전" 처리
        hours_ago = int(date_string.replace("시간전", "").strip())
        return now - timedelta(hours=hours_ago)
    elif "일전" in date_string:
        # "~일전" 처리
        days_ago = int(date_string.replace("일전", "").strip())
        return now - timedelta(days=days_ago)
    else:
        # 절대 날짜 처리 (예: "2025.01.18.")
        return datetime.strptime(date_string, "%Y.%m.%d.")


In [29]:
def is_within_days(date_string, days_to_crawl):
    """
    기사 날짜가 지정된 기간(days_to_crawl) 내에 있는지 확인.
    """
    article_date = parse_date_string(date_string)
    cutoff_date = datetime.now() - timedelta(days=days_to_crawl)
    return article_date >= cutoff_date

In [43]:
def crawl_articles_within_period(driver, url, days_to_crawl):
    """지정된 기간 내의 기사를 크롤링."""
    driver.get(url)
    time.sleep(2)  # 초기 로드 대기
    articles_data = []
    processed_links = set()  # 이미 처리된 기사 링크를 저장하는 집합
    total_articles = 0  # 누적된 기사 수 추적

    while True:
        articles = driver.find_elements(By.CLASS_NAME, "sa_item")
        for article in articles:
            try:
                date_text = article.find_element(By.CLASS_NAME, "sa_text_datetime").text
                if not is_within_days(date_text, days_to_crawl):
                    print(f"{url}: 지정된 기간을 초과한 기사를 발견했습니다. 크롤링 종료.")
                    return articles_data

                link = article.find_element(By.CSS_SELECTOR, "a.sa_text_title").get_attribute("href")
                
                # 중복 기사 확인
                if link in processed_links:
                    continue  # 이미 처리된 기사면 건너뛰기
                
                title = article.find_element(By.CSS_SELECTOR, "strong.sa_text_strong").text
                try:
                    image_url = article.find_element(By.CSS_SELECTOR, "div.sa_thumb img").get_attribute("src")
                except Exception:
                    image_url = None
                summary = article.find_element(By.CLASS_NAME, "sa_text_lede").text
                press = article.find_element(By.CLASS_NAME, "sa_text_press").text

                articles_data.append([link, title, image_url, summary, press, date_text])
                processed_links.add(link)  # 처리된 링크 추가
                total_articles += 1  # 기사 수 누적

            except Exception as e:
                print(f"오류 발생: {type(e).__name__} - {str(e)[:100]}")  # 메시지는 최대 100자만 출력
                continue
        # 진행 상황 로그 출력
        print(f"현재 {total_articles}개의 기사가 크롤링 완료되었습니다.")

        try:
            load_more_button = driver.find_element(By.CSS_SELECTOR, "a.section_more_inner._CONTENT_LIST_LOAD_MORE_BUTTON")
            load_more_button.click()
            time.sleep(2)
        except Exception:
            print(f"{url}: 더 이상 기사가 없습니다. 크롤링 종료.")
            break

    return articles_data

In [48]:
def crawl_multiple_urls(urls, days_to_crawl):
    """여러 URL에서 크롤링."""
    driver = initialize_webdriver()
    all_articles = []

    try:
        for category, url in urls.items():
            print(f"크롤링 시작 - 카테고리: {category}, URL: {url}")
            articles = crawl_articles_within_period(driver, url, days_to_crawl)
            
            # 각 기사에 카테고리 추가
            for article in articles:
                article.insert(0, category)  # 카테고리를 각 기사 데이터의 첫 번째 열에 추가
            
            all_articles.extend(articles)
            print(f"크롤링 완료: {url}, 수집된 기사 수: {len(articles)}")
    finally:
        driver.quit()

    return all_articles

In [47]:
def crawl_articles_with_limit(driver, url, max_articles):
    """
    지정된 개수만큼 기사를 크롤링.
    - max_articles: 크롤링할 최대 기사 수
    """
    driver.get(url)
    time.sleep(2)  # 초기 로드 대기
    articles_data = []
    processed_links = set()  # 이미 처리된 기사 링크를 저장하는 집합
    total_articles = 0  # 누적된 기사 수 추적

    while True:
        articles = driver.find_elements(By.CLASS_NAME, "sa_item")
        for article in articles:
            try:
                # 기사 링크 추출
                link = article.find_element(By.CSS_SELECTOR, "a.sa_text_title").get_attribute("href")
                
                # 중복 기사 확인
                if link in processed_links:
                    continue  # 이미 처리된 기사면 건너뛰기

                # 기사 데이터 추출
                title = article.find_element(By.CSS_SELECTOR, "strong.sa_text_strong").text
                try:
                    image_url = article.find_element(By.CSS_SELECTOR, "div.sa_thumb img").get_attribute("src")
                except Exception:
                    image_url = None
                summary = article.find_element(By.CLASS_NAME, "sa_text_lede").text
                press = article.find_element(By.CLASS_NAME, "sa_text_press").text
                date_text = article.find_element(By.CLASS_NAME, "sa_text_datetime").text

                # 데이터 저장
                articles_data.append([link, title, image_url, summary, press, date_text])
                processed_links.add(link)  # 처리된 링크 추가
                total_articles += 1  # 기사 수 누적

                # 최대 개수에 도달하면 종료
                if total_articles >= max_articles:
                    print(f"최대 {max_articles}개의 기사가 크롤링 완료되었습니다.")
                    return articles_data

            except Exception as e:
                print(f"오류 발생: {type(e).__name__} - {str(e)[:100]}")  # 메시지는 최대 100자만 출력
                continue

        # 진행 상황 로그 출력
        print(f"현재 {total_articles}개의 기사가 크롤링 완료되었습니다.")

        try:
            load_more_button = driver.find_element(By.CSS_SELECTOR, "a.section_more_inner._CONTENT_LIST_LOAD_MORE_BUTTON")
            load_more_button.click()
            time.sleep(2)
        except Exception:
            print(f"{url}: 더 이상 기사가 없습니다. 크롤링을 종료합니다.")
            break

    return articles_data

In [49]:
def crawl_multiple_urls_limit(urls, max_articles):
    """여러 URL에서 크롤링."""
    driver = initialize_webdriver()
    all_articles = []

    try:
        for category, url in urls.items():
            print(f"크롤링 시작 - 카테고리: {category}, URL: {url}")
            articles = crawl_articles_with_limit(driver, url, max_articles)
            
            # 각 기사에 카테고리 추가
            for article in articles:
                article.insert(0, category)  # 카테고리를 각 기사 데이터의 첫 번째 열에 추가
            
            all_articles.extend(articles)
            print(f"크롤링 완료: {url}, 수집된 기사 수: {len(articles)}")
    finally:
        driver.quit()

    return all_articles

In [32]:
def save_to_csv(filename, data, fields):
    """데이터를 CSV 파일로 저장."""
    try:
        with open(filename, mode='w', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(fields)  # 헤더 작성
            writer.writerows(data)
        print(f"CSV 파일 '{filename}'에 저장 완료!")
    except Exception as e:
        print(f"CSV 저장 중 오류 발생: {e}")

In [50]:
output_file = "dataset_news_data.csv"
fields = ["카테고리", "기사 링크", "기사 제목", "이미지 URL", "기사 요약", "언론사", "게시 날짜"]

all_articles = crawl_multiple_urls_limit(urls, 1000)
save_to_csv(output_file, all_articles, fields)

크롤링 시작 - 카테고리: 금융, URL: https://news.naver.com/breakingnews/section/101/259
현재 36개의 기사가 크롤링 완료되었습니다.
현재 72개의 기사가 크롤링 완료되었습니다.
현재 108개의 기사가 크롤링 완료되었습니다.
현재 144개의 기사가 크롤링 완료되었습니다.
현재 180개의 기사가 크롤링 완료되었습니다.
현재 216개의 기사가 크롤링 완료되었습니다.
현재 252개의 기사가 크롤링 완료되었습니다.
현재 288개의 기사가 크롤링 완료되었습니다.
현재 324개의 기사가 크롤링 완료되었습니다.
현재 360개의 기사가 크롤링 완료되었습니다.
현재 396개의 기사가 크롤링 완료되었습니다.
현재 432개의 기사가 크롤링 완료되었습니다.
현재 468개의 기사가 크롤링 완료되었습니다.
현재 504개의 기사가 크롤링 완료되었습니다.
현재 540개의 기사가 크롤링 완료되었습니다.
현재 576개의 기사가 크롤링 완료되었습니다.
현재 612개의 기사가 크롤링 완료되었습니다.
현재 648개의 기사가 크롤링 완료되었습니다.
현재 684개의 기사가 크롤링 완료되었습니다.
현재 720개의 기사가 크롤링 완료되었습니다.
현재 756개의 기사가 크롤링 완료되었습니다.
현재 792개의 기사가 크롤링 완료되었습니다.
현재 828개의 기사가 크롤링 완료되었습니다.
현재 864개의 기사가 크롤링 완료되었습니다.
현재 900개의 기사가 크롤링 완료되었습니다.
현재 936개의 기사가 크롤링 완료되었습니다.
현재 972개의 기사가 크롤링 완료되었습니다.
최대 1000개의 기사가 크롤링 완료되었습니다.
크롤링 완료: https://news.naver.com/breakingnews/section/101/259, 수집된 기사 수: 1000
크롤링 시작 - 카테고리: 증권, URL: https://news.naver.com/breakingnews/section/101/258
현재 36개의 기사가 크롤링 완료되었습니다.
현재 72개의 기사가 크롤링 완료되었

In [54]:
import csv

def extract_category_and_links_from_csv(file_name, category_column_name, link_column_name):
    """
    CSV 파일에서 '카테고리'와 '기사 링크' 열의 데이터를 추출
    :param file_name: CSV 파일 이름
    :param category_column_name: 카테고리가 저장된 열 이름
    :param link_column_name: 링크가 저장된 열 이름
    :return: (카테고리, 링크) 튜플 리스트
    """
    data = []

    # CSV 파일 열기
    with open(file_name, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)

        # 각 행에서 '카테고리'와 '기사 링크' 데이터 추출
        for row in reader:
            if category_column_name in row and link_column_name in row:
                data.append((row[category_column_name], row[link_column_name]))
    
    return data

# CSV 파일에서 '카테고리'와 '기사 링크' 추출
file_name = "dataset_news_data.csv"  # 기존 저장된 CSV 파일
category_column_name = "카테고리"  # 카테고리가 저장된 열 이름
link_column_name = "기사 링크"  # 링크가 저장된 열 이름
category_and_links = extract_category_and_links_from_csv(file_name, category_column_name, link_column_name)

# 결과 확인
print(f"추출된 데이터 ({len(category_and_links)}개):")
for category, link in category_and_links[:5]:  # 첫 5개 데이터만 출력
    print(f"카테고리: {category}, 기사 링크: {link}")

추출된 데이터 (7837개):
카테고리: 금융, 기사 링크: https://n.news.naver.com/mnews/article/009/0005432123
카테고리: 금융, 기사 링크: https://n.news.naver.com/mnews/article/087/0001093852
카테고리: 금융, 기사 링크: https://n.news.naver.com/mnews/article/087/0001093851
카테고리: 금융, 기사 링크: https://n.news.naver.com/mnews/article/087/0001093850
카테고리: 금융, 기사 링크: https://n.news.naver.com/mnews/article/087/0001093849


In [52]:
# CSS 선택자를 사용해 요소의 텍스트 가져오기
def fetch_element_text(driver, css_selector, error_message):
    try:
        return driver.find_element(By.CSS_SELECTOR, css_selector).text.strip()
    except Exception as e:
        print(f"{error_message}: {e}")
        return None

# CSS 선택자를 사용해 요소의 속성 값 가져오기
def fetch_element_attribute(driver, css_selector, attribute, error_message):
    try:
        return driver.find_element(By.CSS_SELECTOR, css_selector).get_attribute(attribute)
    except Exception as e:
        print(f"{error_message}: {e}")
        return None


In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time


# WebDriver 초기화 및 페이지 열기
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

output_data = []
fields = ["카테고리", "뉴스 제목", "뉴스 내용", "뉴스 링크"]
cnt = 0 # 크롤링 성공 카운트

try:
    for category, link in category_and_links:
        try:
            # 페이지 열기
            driver.get(link)

            # JavaScript 로드 대기
            time.sleep(2)
            
            # 주요 기사 정보 가져오기
            title = fetch_element_text(driver, "#title_area", "뉴스 제목을 가져오는 데 실패했습니다.")
            content = fetch_element_text(driver, "#dic_area", "뉴스 내용을 가져오는 데 실패했습니다.")

            # 기사 데이터 저장
            article_data = [
                category, title, content, link
            ]
            output_data.append(article_data)

            # 성공적으로 크롤링한 경우 카운트 증가 및 로그 출력
            cnt += 1
            print(f"({cnt}) 크롤링 완료: {link}")

        except Exception as e:
            print(f"URL 처리 중 오류 발생: {e}")
            continue

finally:
    # 브라우저 종료
    driver.quit()

    
# CSV 파일로 저장
output_file = "dataset_news_data_detailed.csv"
save_to_csv(output_file, output_data, fields)

print(f"CSV 파일 '{output_file}'에 저장 완료!")

(1) 크롤링 완료: https://n.news.naver.com/mnews/article/009/0005432123
(2) 크롤링 완료: https://n.news.naver.com/mnews/article/087/0001093852
(3) 크롤링 완료: https://n.news.naver.com/mnews/article/087/0001093851
(4) 크롤링 완료: https://n.news.naver.com/mnews/article/087/0001093850
(5) 크롤링 완료: https://n.news.naver.com/mnews/article/087/0001093849
(6) 크롤링 완료: https://n.news.naver.com/mnews/article/009/0005432116
(7) 크롤링 완료: https://n.news.naver.com/mnews/article/277/0005535703
(8) 크롤링 완료: https://n.news.naver.com/mnews/article/015/0005084677
(9) 크롤링 완료: https://n.news.naver.com/mnews/article/014/0005298411
(10) 크롤링 완료: https://n.news.naver.com/mnews/article/658/0000095004
(11) 크롤링 완료: https://n.news.naver.com/mnews/article/018/0005928729
(12) 크롤링 완료: https://n.news.naver.com/mnews/article/018/0005928728
(13) 크롤링 완료: https://n.news.naver.com/mnews/article/018/0005928727
(14) 크롤링 완료: https://n.news.naver.com/mnews/article/018/0005928726
(15) 크롤링 완료: https://n.news.naver.com/mnews/article/018/0005928725
(16)

In [56]:
import csv
from collections import Counter

# 파일 이름과 헤더 이름 설정
file_name = "dataset_news_data_detailed.csv"  # CSV 파일 이름
category_column_name = "카테고리"  # 카테고리 열 이름

# 카테고리별 뉴스 개수를 저장할 Counter 객체 생성
category_counts = Counter()

# CSV 파일 읽기
with open(file_name, mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)

    # 각 행에서 카테고리 데이터 추출 및 카운트
    for row in reader:
        category = row.get(category_column_name)  # 카테고리 열에서 값 가져오기
        if category:  # 값이 있는 경우만 카운트 증가
            category_counts[category] += 1

# 결과 출력
print("카테고리별 뉴스 개수:")
for category, count in category_counts.items():
    print(f"{category}: {count}개")

카테고리별 뉴스 개수:
금융: 1000개
증권: 1000개
산업/재계: 1000개
중기/벤처: 1000개
부동산: 1000개
글로벌 경제: 837개
생활경제: 1000개
경제일반: 1000개
